This is just another "test" sent by a recruiter agency which I share here so that they will have to invent another one.

https://raw.githubusercontent.com/frgomes/jupyter-notebooks/master/NowTV%20-%20Puzzle%20Solver.ipynb

The Problem
======

Word puzzle solver
-------

Some newspapers have a word problem where nine characters are displayed and you form as many words as possible from these letters.  You must use the highlighted letter.

For example:

http://nineletterword.tompaton.com/

 

For our problem, you are given a nine character string and need to return a list of strings containing all words you can form, which must also contain the first character of the input string.  The words should check against an online word list such as:

https://raw.githubusercontent.com/dwyl/english-words/master/words.txt

 

The starting method signature is:

 
```scala
def solutions(in: String): List[String] = ???
```

----

Solution
====

The solution is presented as a Jupyter notebook, a prototyping tool very valuable for testing ideas, validating what you are doing, understanding pros and cons, perform performance measurements, take a look at data structures, etc... before you write any code into the IDE.

What about test cases?
----

The code presented here works. It was already tested interatively and bugs were already fixed.

It does not mean that an application meant to be put in production does not need test cases. On the contrary. After moving the code snippets from here to the IDE, you will have to write test cases as usual, make sure the code builds, all tests pass under Jenkins, etc.

The main difference is: you write code **after**, not *before* you arrive to a conceptual solution, not *before* you are confident you are in the right track.

It simply does not make sense starting writing tests before thinking about the algorithm, performance concerns and data structures involved.

What about TDD?
----

I'm not against TDD.

However, TDD does not imply necessarily writing tests before you have a reasonable good idea about how your application should look like.

TDD *before* you understand the problem domain and how the application should look like implies that you arrive to tests which probably work pretty well ... but later you realize that you are solving the overall problem pretty badly, which means that your application demands reengineering.

In other words: rework.

Rework takes time and costs money. You'd better understand the problem domain pretty well in the beginning, when you don't have anything written and it's pretty easy to crunch a piece of paper with bad ideas, throw it away and start over fresh with brand new piece of paper.

----

Dictionary words
------------------
This is a dataset consisting of 350,000+ words apparently collected from a number of documents not known at this time. Several words are not really words but parts of words such as *'s*, *'ll*, *'mongst*. Also, there are numbers like *2*, *1080*, *3rd*. Also, there are expressions like *ahh*. So, a data cleansing would be probably a good idea here.

However, for the sake of this exercise, we simply consider all entries of the dictionary as valid entries.

In [1]:
val source = scala.io.Source.fromURL("https://raw.githubusercontent.com/dwyl/english-words/master/words.txt")

source: io.BufferedSource = non-empty iterator

Hashtables and prime numbers
--------------------------------
In order to expedite access to the dictionary, we calculate a hash code for a given word and we insert that word into a balanced tree.

Given that there are 350,000+ words in the dictionary, we've chosen a prime number so that we expect that each balanced tree will have up to 20 elements or so, which means that we can arrive to the word we are looking for in up to 5 comparisons, supported by a tree of up to 32 elements.

There's no really guarantee that all trees will have up to 32 elements. This is just an expectation which may or may not turn to be true. After building the trees we check what would be the most populated tree, in order to double check if the prime number we've chosen seems to be a reasonable choice.

Hashing and hashing again
-----------------------------
To be more precise, we find the ``hashCode`` (let's call it simply ``hash``), we divide it by a prime number, finding a another hash which is more manageable (let's call it ``tinyHash``).

```scala
val hash     = word.hashCode
val tinyHash = hash % prime
```

This is done this way since we will keep a reasonably small data structure indexed by ``tinyHash``. Each entry in this data structure is a balanced tree which contains words hashed by their true hash code, i.e.: ``hash``. This way we try to avoid collisions.

Balanced trees and binary search
-------------------------------------

Despite our efforts to avoid collisions in the previous step, there's no really guarantee that we will definitely avoid collisions at all times. If we fail to do that, the last update wins, which means that only the last word inserted will be present in the tree, since it overwrites all other words with the very same hash code.

How we can circumvent this problem?

One answer would be employing a balanced tree which supports duplicates. Another way would be employing some other data structure which behaves well in the presence of duplicates.

So, instead of a balance tree, we will be simply storing an ordered list of words. Given that this data structure is ordered, we can then employ a **binary search** in order to arrive to a possible match. The number of comparisons of a binary search is (not by coincidence!) the same number of comparisons we would observe if we were employing a balanced tree.

In [2]:
val prime = 16381

prime: Int = 16381

In [3]:
val lines: Seq[String] = source.getLines.toList

lines: Seq[String] = List(
  "&c",
  "'d",
  "'em",
  "'ll",
  "'m",
  "'mid",
  "'midst",
  "'mongst",
  "'prentice",
  "'re",
  "'s",
  "'sblood",
  "'sbodikins",
  "'sdeath",
  "'sfoot",
  "'sheart",
  "'shun",
  "'slid",
  "'slife",
...

In [4]:
val dict = lines.groupBy(line => (line.hashCode%prime).abs).mapValues(_.sorted.toArray)

dict: Map[Int, Array[String]] = Map(
  14221 -> Array(
    "boarding",
    "bocal",
    "equibalanced",
    "eudaemonic",
    "everglade",
    "gyrfalcon",
    "hierophanticly",
    "horizon's",
    "inbits",
    "khahoon",
    "peritonsillar",
    "preconversational",
    "primuses",
    "realive",
    "reprehensibly",
    "rhinocerotine",
    "summonable",
    "swellfishes",
...

Verifying tree depth
-----------------------
We've chosen the prime number so that we would expect up to 5 comparisons, i.e.: a tree with up to 32 elements. Lets verify what would be the maximum number of elements we've got. If it is less than or equal to 32, we are doing things according to plan.

In [5]:
dict.mapValues(_.size).map{ case (k,v) => v }.reduceLeft(_ max _)

res4: Int = 45

A bit off track
-----------------
The maximum tree depth in this case implies on a maximum of 6 comparisons. A bit higher than we would expect but still not bad at all.

Given any word, we find its *conceptual* balanced tree (actually an array which we find information employing a binary search) in constant time. After that, we do a maximum of 6 comparisons. It's not bad at all.

Obviously we can make it better if performance happens to be an issue. In order to do that, we need to try other prime numbers. I will let this exercise for the reader.

Ability to find words
---------------------
OK. Now we need the ability to find words in the dictionary.

Given a certain ``String``, we need to find its ``hash`` and its ``tinyHash``. The ``tinyHash`` is employed so that we can find the *conceptual* balanced tree (implemented as an ordered ``Array[String]``). Found that, we then perform a binary search.

The binary search will find words which match the ``String`` we have at hand, in case there's such word in the dictionary. We will not use ``hash`` directly, despite that the algorithm which performs the binary search may or may not employ the very same concept.

OK. First of all, we need to wrap ``java.util.Array#binarySearch`` into something a bit more convenient:

In [6]:
class RichArray[T <: AnyRef](a: Array[T]) { 
   def binarySearch(key: T) = {
     java.util.Arrays.binarySearch(a.asInstanceOf[Array[AnyRef]],key)
   }
}
implicit def richArray[T <: AnyRef](a: Array[T]) = new RichArray(a)


defined class RichArray
defined function richArray

Finding one word or two
-----------------------
Now let's exercise the idea.

Given a certain ``String``, we find the *conceptual balanced tree* (which is actually implemented as an ordered ``Array[String]``) and we perform a binary search.

If we find a positive index, it's in the dictionary. If we find a negative index, it's not.

In [7]:
{
    val word = "resilient"
    val tinyHash = (word.hashCode % prime).abs
    val index = dict(tinyHash).binarySearch(word)
    val found = index >= 0
}

word: String = "resilient"
tinyHash: Int = 15619
index: Int = 10
found: Boolean = true

In [8]:
{
    val word = "linux"
    val tinyHash = (word.hashCode % prime).abs
    val index = dict(tinyHash).binarySearch(word)
    val found = index >= 0
}

word: String = "linux"
tinyHash: Int = 6814
index: Int = -13
found: Boolean = false

Sizing the problem
---------------------

OK. Now that we know that we can find words in the dictionary limited by a higher boundary of 6 comparisons, it's time to think about how we can find the words from the puzzle in the dictionary.

The problem is: we don't have words in the puzzle: we have just a certain quantity of letters which may be eventually a dictionary word. This is not really a problem, since we can determine if a candidate string is a dictionary word in just 6 comparisons. The problem is the size of the problem.

The problem is that we have up to 9 letters as a candidate word and we can shuffle these letters any way we wish; we can also remove letters and shuffle again. The problem is that the size of the problem is *roughly* P(9,8) + P(9,7) + ... + P(9,3) + P(9,2) where P(n,m) represents the permutation of *n* letters grouped by *m*. This is a big number. P(9,8) is ~43 million... so we don't even need to finish the entire calculation to realize that performing these sort of permutations **is not** the way to go.


A tentative approach
-----------------------
What if we do not make any permutations at all? We could simply consider a certain set of letters, regardless their relative order.

Now the problem reduces to the ability to find in the dictionary those words which share the same properties of that set of letters we have at hand, regardless their relative order.

We could also try to reduce the problem given the number of letters we have. Since we've selected 5 letters, we can be sure that dictionary words made of 4 letters are not good candidates.

OK. The idea is: let's calculate a relatively naive hash function and attach information about the number of letters we are interested. For example (and simplistically):

```scala
val naiveHash = s(0) + s(1) + ... + s(n-1)
val naiveHashPlusSize = naiveHash * 10 + (n%10)
```

OK. Now we have to calculate ``naiveHashPlusSize`` for every word in the dictionary and create another data structure which classfies words according to this criteria.


Refining our plan
--------------------

So, the idea is now calculate the ``naiveHashPlusSize`` for a candidate word from the puzzle and find a list of dictionary words which match the same ``naiveHashPlusSize``. Sounds good. But, how many dictionary words we are really talking about?

Well... it depends on the number of entries in the hashtable and their statistic distribution. We don't really know this information at this point. Let's simply try this idea and see if we obtain a data structure which looks to be reasonable, in other words: there's a relatively manageable number of words sharing the same ``naiveHashPlusSize``.

In [9]:
def naiveHashPlusSize(s: String): Int = {
    (s.map(c => c - ' ').sum * 10) + (s.length % 10)
}

defined function naiveHashPlusSize

In [10]:
val dict2 = lines.groupBy(line => naiveHashPlusSize(line)).mapValues(_.sorted.toArray)

dict2: Map[Int, Array[String]] = Map(
  2163 -> Array(
    "abu",
    "act",
    "ads",
    "aer",
    "aho",
    "ain",
    "alk",
    "ani",
    "are",
    "ava",
    "bim",
    "bog",
    "cat",
    "cep",
    "chm",
    "cli",
    "crc",
    "das",
...

In [11]:
dict2.mapValues(_.size).map{ case (k,v) => v }.reduceLeft(_ max _)

res10: Int = 1095

Looks relatively well
---------------------
In a nutshell, it means that we will do a maximum of ~1100 comparisons in the worst case.

This is definitely better than a full table scan as per
https://github.com/dwyl/autocomplete/blob/master/index.js

More refinements?
-----------------

There's definitely room for more refinements.

The way it is at the moment, for every word in our data structure (here called ``dict2``) we will have to compare if shuffling this word eventually arrives to the candidate word we have at hand.

Actually, it's easier to do something different: we sort the candidate word we have at hand and we sort the word obtained from ``dict2`` and we see whether they match or not. Something like this:

```scala
if(candidate.sorted == word.sorted) ... // we've found something here!
```

We will have to do this ~1100 times in the worst case, every time a new candidate word is entered. It would be nice if we had a binary search here too. Employing a binary search, we reduce the number of comparisons from ~1100 to only ~10 comparisons.

Given a candidate word, we sort its component letters and we try to find it in the hashtable which, not by coincidence, must have dictionary words already sorted too.

To be more precise, we actually have to keep both: we need to find dictionary words via its sorted representation and, after that, we need to return the original representation, as plain text, exactly as provided in the input source.

Since dictionary words may collide after sorted, we need to actually store a hashtable inside a hashtable. There's simply no way to escape this fact, even if we find a bigger prime number as divisor, even if we do not divide the calculated hash by any prime number at all.

The data structure consists of a hashtable or hashtables, as shown below:

In [28]:
Seq("faca", "cafa", "jacu", "cuja", "abb", "bba", "aba", "aabb", "abba", "bbaa")
    .groupBy(line => line.hashCode.abs%2)
    .mapValues(words => 
                 words
                   .map(word => (word.sorted -> word))
                   .groupBy(_._1).map { case (k,v) => (k,v.map(_._2))})

res27: Map[Int, Map[String, Seq[String]]] = Map(
  1 -> Map(
    "acju" -> List("jacu", "cuja"),
    "aacf" -> List("faca", "cafa"),
    "abb" -> List("abb", "bba")
  ),
  0 -> Map("aabb" -> List("aabb", "abba", "bbaa"), "aab" -> List("aba"))
)

Now let's define ``dict3``, which hopefully is our final version of the most important data structure we need to solve this exercise.

In [13]:
val dict3 =
  lines
    .groupBy(line => naiveHashPlusSize(line))
    .mapValues(words => 
                 words
                   .map(word => (word.sorted -> word))
                   .groupBy(_._1).map { case (k,v) => (k,v.map(_._2))})

dict3: Map[Int, Map[String, Seq[String]]] = Map(
  2163 -> Map(
    "aho" -> List("aho", "hao"),
    "cil" -> List("cli"),
    "dgm" -> List("mgd"),
    "abu" -> List("abu"),
    "cep" -> List("cep"),
    "een" -> List("een", "nee"),
    "bgo" -> List("bog", "gob"),
    "bim" -> List("bim", "ibm", "mib"),
    "egl" -> List("gel", "leg"),
    "afq" -> List("faq", "qaf"),
    "ajm" -> List("jam"),
    "ads" -> List("ads", "das", "sad"),
    "ain" -> List("ain", "ani"),
    "dik" -> List("kid"),
    "chm" -> List("chm"),
    "aav" -> List("ava"),
    "dhl" -> List("hld", "lhd"),
    "efm" -> List("emf", "fem"),
...

Ability to match candidate words
-------------------------------------

OK. Now we are ready to enter some sort of random text and see if we find dictionary words for it. Let's try a couple of words and see how it behaves.

In [14]:
def findWords(candidate: String): Seq[String] = {
    val hash = naiveHashPlusSize(candidate)
    val sorted = candidate.sorted
    val matches = 
      dict3
        .getOrElse(hash, Map.empty[String, Seq[String]])
        .getOrElse(sorted, Seq.empty[String])
        .filter(word => sorted == word.sorted)
    matches
}

defined function findWords

In [15]:
findWords("drst")

res14: Seq[String] = List()

In [16]:
findWords("evarega")

res15: Seq[String] = List("average")

In [17]:
findWords("resliient")

res16: Seq[String] = List("resilient")

Ability to find all candidate substrings of candidate word
------------------------------------------------------------------

Now, all we need to do is the ability to find all substrings of a candidate word, not forgetting that the **first letter** must be always present.

The idea is that we find all substrings of a candidate word *except the first letter*, then we add the first letter later to all positions it would be necessary.

But wait! We will sort the candidate word (or candidate substring of it) anyway. So, it does not matter. We can simply add the first letter to the beginning and we are done. Also, we don't need to care about relative order of characters, since we are going to sort letters anyway, the same way we sort dictionary words when we insert them into ``dict3``.

So, below we demonstrate how it would work. Suppose the word "darts". We remove "d" and we obtain a list of substrings from "arts", like shown below:

In [18]:
def parts(s: String): Seq[String] = {
    s.size match {
        case 0 => Seq.empty[String]
        case 1 => Seq(s(0).toString)
        case _ => 
            s.substring(1).inits.flatMap(_.tails.toList.init).toSeq
              .map(text => s(0) + text)
              .toSet
              .toList
    }
}

defined function parts

In [19]:
parts("darts")

res18: Seq[String] = List("drts", "dt", "darts", "ds", "dart", "drt", "da", "dr", "dts", "dar")

In [20]:
"Antidisestablishmentarianism".toLowerCase

res19: String = "antidisestablishmentarianism"

Putting it all together
-----------------------

Now we are ready to arriving to a solution.

In [21]:
def solutions(in: String): Seq[String] = {
    parts(in.toLowerCase).flatMap(candidate => findWords(candidate))
}

defined function solutions

Doing a couple of performance tests
-----------------------------------
Let's employ 9 characters as the specification says. But let's also play with a bit more.

In [22]:
def test(letters: String): (Seq[String], Long) = {
    val before = new java.util.Date().getTime()
    val result = solutions(letters)
    val after  = new java.util.Date().getTime()
    val elapsed = after - before
    (result, elapsed)
}

defined function test

In [23]:
{
    val (result, milliseconds) = test("aimlessly")
}

result: Seq[String] = List(
  "lass",
  "sals",
  "ae",
  "ea",
  "am",
  "ma",
  "massel",
  "lyssa",
  "slays",
  "aly",
  "lay",
  "lays",
  "slay",
  "alem",
  "alme",
  "amel",
  "lame",
  "leam",
  "male",
...
milliseconds: Long = 44L

In [24]:
{
    val (result, milliseconds) = test("confirmed")
}

result: Seq[String] = List(
  "dec",
  "con",
  "nco",
  "crime",
  "merci",
  "co",
  "oc",
  "cfi",
  "cif",
  "cir",
  "confirmed",
  "cd",
  "dc",
  "merc",
  "crim",
  "cr",
  "rc",
  "conf",
  "ce",
...
milliseconds: Long = 18L

In [25]:
{
    val (result, milliseconds) = test("performance")
}

result: Seq[String] = List(
  "mp",
  "pm",
  "fp",
  "pf",
  "perf",
  "pref",
  "ap",
  "pa",
  "per",
  "pre",
  "rep",
  "profer",
  "profre",
  "np",
  "rpm",
  "ep",
  "pe",
  "fop",
  "encamp",
...
milliseconds: Long = 41L

In [26]:
{
    val (result, milliseconds) = test("development")
}

result: Seq[String] = List(
  "del",
  "eld",
  "led",
  "delve",
  "devel",
  "dev",
  "loved",
  "voled",
  "dt",
  "deve",
  "do",
  "od",
  "moped",
  "loped",
  "poled",
  "dem",
  "med",
  "pendom",
  "vd",
...
milliseconds: Long = 33L

In [27]:
{
    val (result, milliseconds) = test("Antidisestablishmentarianism")
}

result: Seq[String] = List(
  "antra",
  "ratan",
  "alba",
  "baal",
  "lamish",
  "shimal",
  "anatira",
  "ishmael",
  "asse",
  "seas",
  "ae",
  "ea",
  "ament",
  "manet",
  "meant",
  "menat",
  "menta",
  "teman",
  "sidia",
...
milliseconds: Long = 155L

Conclusion
----------

Access time around ~20ms for candidate words of 9 letters looks pretty good.

The first part of the exploration was not really used in the final solution, but helped as an explorarion of the problem domain and, in particular, in regards to performance issues.

But... if the input source was already sorted (or at least apparently sorted or apparently partially sorted)... why then creating expending extra time loading a relatively complex data structure? Couldn't the idea of a binary search be applicable straight away to a large ``Array[String]`` which contains all dictionary words?

The short answer is: if you are willing to perform just a couple of queries... yes. However, if you are willing to provide a service which performs well under load then, in this case, performance is key and every millisecond counts.

And there's still more room for optimization.
